In [8]:
import pandas as pd
import altair as alt

df = pd.read_csv('oscars_limpio_1997_2025-4.csv')
streaming_keywords = ['netflix', 'amazon', 'prime video', 'apple', 'hulu', 'disney+']

def is_streaming(row):
    combined_distributors = ' '.join([
        str(row['distribuidor_1']), str(row['distribuidor_2']),
        str(row['distribuidor_3']), str(row['distribuidor_4']),
        str(row['distribuidor_5'])
    ]).lower()
    return any(keyword in combined_distributors for keyword in streaming_keywords)

df['is_streaming'] = df.apply(is_streaming, axis=1)

summary = df.groupby(['Year', 'is_streaming'])['winner'].agg(['count', 'sum']).reset_index()
summary = summary.rename(columns={'count': 'nominations', 'sum': 'wins'})

summary_long = summary.melt(
    id_vars=['Year', 'is_streaming'],
    value_vars=['nominations', 'wins'],
    var_name='Metric',
    value_name='Count'
)

summary_long['Distribuidor'] = summary_long['is_streaming'].map({True: 'Streaming', False: 'Tradicional'})
summary_long['Métrica'] = summary_long['Metric'].map({'nominations': 'Nominaciones', 'wins': 'Premios Ganados'})

color_scale = alt.Scale(domain=['Streaming', 'Tradicional'], range=['#e63946', '#457b9d'])

base = alt.Chart(summary_long).mark_line(point=alt.OverlayMarkDef(filled=True, size=60)).encode(
    x=alt.X('Year:O', title='Año'),
    y=alt.Y('Count:Q', title='Cantidad', axis=alt.Axis(labelFontSize=12, titleFontSize=14)),
    color=alt.Color('Distribuidor:N', title='Tipo de Distribuidor', scale=color_scale),
    strokeDash=alt.StrokeDash('Distribuidor:N', legend=None),
    tooltip=['Year', 'Distribuidor', 'Métrica', 'Count']
).properties(
    width=320,
    height=260
)

chart = base.facet(
    column=alt.Column('Métrica:N', title=None, header=alt.Header(labelFontSize=14, labelFontWeight='bold'))
).properties(
    title={
        "text": ["Evolución de la presencia de plataformas de streaming en los Premios Oscar"],
        "subtitle": ["Comparación de nominaciones y premios ganados frente a estudios tradicionales (1997–2025)"],
        "fontSize": 18,
        "subtitleFontSize": 14,
        "anchor": "start"
    }
).configure(
    background='#ffffff'
).configure_view(
    stroke=None
).configure_axis(
    labelFontSize=11,
    titleFontSize=13
).configure_legend(
    labelFontSize=12,
    titleFontSize=13
)

chart.save('streaming_vs_tradicional.html')
chart.save('streaming_vs_tradicional.html')
files.download('streaming_vs_tradicional.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>